In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.0/112.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.7 requires tyro, which is not installed.
unsloth-zoo 2025.3.7 requires protobuf<4.0.0,

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = ["unsloth/llama-3-8b-bnb-4bit"]

instruction_title = "DESCRIBE THIS AMAZON SEARCH"


DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/data/trn.json' #Local do trn.json
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/data/trn-format.json" #Local que o trn-format.json será criado

data = []
count = 0

data_rows = 10000 #Alterar numero de linhas do json

with open(DATA_PATH, encoding='utf-8') as rows:
  for i in rows:
    if count < data_rows:
      data.append(json.loads(i))
      count += 1

def format_dataset_into_model_input(data):
  inputs = []
  outputs = []

  for i in data:
    inputs.append(i['title'])
    outputs.append(i['content'])

  formatted_data = {
      "input": inputs,
      "output": outputs
  }

  title_data = inputs

  with open(OUTPUT_PATH_DATASET, 'w') as output_file:
      json.dump(formatted_data, output_file, indent=4)

  return title_data

In [5]:
title_data = format_dataset_into_model_input(data)
data = ['']
i = ''

In [6]:
from unsloth import FastLanguageModel, is_bfloat16_supported

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [7]:
model = FastLanguageModel.get_peft_model(
  model,
  r = 16,
  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
  lora_alpha = 16,
  lora_dropout = 0,
  bias = "none",

  use_gradient_checkpointing = "unsloth",
  random_state = 3407,
  use_rslora = False,
  loftq_config = None,
)

Unsloth 2025.3.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
  instructions = instruction_title
  inputs       = examples["input"]
  outputs      = examples["output"]
  texts = []
  for instruction, input, output in zip(instructions, inputs, outputs):

      text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
      texts.append(text)
  return { "text" : texts, }
pass

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True, batch_size=8)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
  model = model,
  tokenizer = tokenizer,
  train_dataset = dataset,
  dataset_text_field = "text",
  max_seq_length = max_seq_length,
  dataset_num_proc = 2,
  packing = False,
  args = TrainingArguments(
      per_device_train_batch_size = 2,
      gradient_accumulation_steps = 4,
      warmup_steps = 5,
      max_steps = 10,
      learning_rate = 2e-4,
      fp16 = not is_bfloat16_supported(),
      bf16 = is_bfloat16_supported(),
      logging_steps = 1,
      optim = "adamw_8bit",
      weight_decay = 0.01,
      lr_scheduler_type = "linear",
      seed = 3407,
      output_dir = "outputs",
      report_to="none",
  ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/4,582,543,360 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.188700
2,2.726400
3,2.343100
4,2.567500
5,2.789200
6,2.598000
7,2.625500
8,1.915100
9,2.089500
10,2.047400


In [11]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction_title,
        "Girls Ballet Tutu Neon Pink",
        "High quality 3 layer ballet tutu. 12 inches in length",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
DESCRIBE THIS AMAZON SEARCH

### Input:
Girls Ballet Tutu Neon Pink

### Response:
High quality 3 layer ballet tutu. 12 inches in length. Available in 6 colors. 100% polyester.<|end_of_text|>


In [12]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/data/trn_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/data/trn_model")

('/content/drive/MyDrive/data-challenge-3/trn_model/tokenizer_config.json',
 '/content/drive/MyDrive/data-challenge-3/trn_model/special_tokens_map.json',
 '/content/drive/MyDrive/data-challenge-3/trn_model/tokenizer.json')

In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = "/content/drive/MyDrive/Colab Notebooks/data/trn_model",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [14]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[alpaca_prompt.format(
  instruction_title,
  title_data[0],
  "",

)], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
DESCRIBE THIS AMAZON SEARCH

### Input:
Girls Ballet Tutu Neon Pink

### Response:
Girls Ballet Tutu Neon Pink. This is a neon pink ballet tutu for girls. It has a stretchy waistband that is elasticized and is made of a nylon/spandex blend. The tutu is approximately 12 inches long. It is made in the USA. It is available in sizes 3T-8. It is available in sizes 3T-8. The tutu is made of a nylon/spandex blend. It is made of a nylon/spandex blend. It is available in sizes 3T-8. It is available in sizes 3T-8. The tutu is made of
